# Django tutorial model experiments

## Setup

In [1]:
# jupyter django setup
import os,django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

# notebook setup
from texts.models import Author, Text
import requests
def getlink(url): return requests.get(url).text

## Create objects

### Authors

Self-explanatory I suppose.

In [2]:
Author._meta.get_fields()

(<ManyToOneRel: texts.work>,
 <ManyToOneRel: texts.book>,
 <ManyToOneRel: texts.text>,
 <django.db.models.fields.BigAutoField: id>,
 <django.db.models.fields.CharField: name>,
 <django.db.models.fields.IntegerField: dob>)

In [3]:
# Create author
shelley, created = Author.objects.get_or_create(name="Mary Wollstonecraft Shelley", dob=1797)
shelley, created

(<Author #1: Mary Wollstonecraft Shelley>, True)

### Works

The abstract 'work' of _Frankenstein_, no matter its 1818 or 1831 editions or books.

In [4]:
Work._meta.get_fields()

(<ManyToOneRel: texts.book>,
 <ManyToOneRel: texts.text>,
 <django.db.models.fields.BigAutoField: id>,
 <django.db.models.fields.related.ForeignKey: author>,
 <django.db.models.fields.CharField: title>)

In [5]:
work, created = Work.objects.get_or_create(title="Frankenstein", author=shelley)
work, created

(<Work #1: Frankenstein, by <Author #1: Mary Wollstonecraft Shelley>>, True)

### Books

The actually published material artifacts of books are marked here. These can and should have an author; they can be associated with a work, or not.

In [6]:
book1, created = Book.objects.get_or_create(title="Frankenstein", year=1818, work=work, author = shelley)
book1, created

(<Book #1: 'Frankenstein', in <Work #1: Frankenstein, by <Author #1: Mary Wollstonecraft Shelley>>>,
 True)

In [7]:
book2, created = Book.objects.get_or_create(title="Frankenstein", year=1831, work=work, author = shelley)
book2, created

(<Book #2: 'Frankenstein', in <Work #1: Frankenstein, by <Author #1: Mary Wollstonecraft Shelley>>>,
 True)

### Texts

An actual text string, which can be associated with a book, or just a work, or a separate or additional author, etc.

In [8]:
text1, created = Text.objects.get_or_create(
    txt = getlink('https://www.gutenberg.org/cache/epub/41445/pg41445.txt'), 
    book = book1
)
text1, created

(<Text #1: '\ufeffThe Project Gutenberg eB ...' (435631 chars), in <Book #1: 'Frankenstein', in <Work #1: Frankenstein, by <Author #1: Mary Wollstonecraft Shelley>>>>,
 True)

In [9]:
text2, created = Text.objects.get_or_create(
    txt = getlink('https://www.gutenberg.org/cache/epub/42324/pg42324.txt'), 
    book = book1
)
text2, created

(<Text #2: '\ufeffThe Project Gutenberg EB ...' (465593 chars), in <Book #1: 'Frankenstein', in <Work #1: Frankenstein, by <Author #1: Mary Wollstonecraft Shelley>>>>,
 True)

In [10]:
# Get author even if none set on text object directly
text1.author, text1.get_author()

(None, <Author #1: Mary Wollstonecraft Shelley>)

In [12]:
# Text objects equate but are not identical
idx = text1.id
(
    Text.objects.get(id=idx) is text1, 
    Text.objects.get(id=idx) == text1,
    Text.objects.get(id=idx) is Text.objects.get(id=idx),
    Text.objects.get(id=idx) == Text.objects.get(id=idx),
)

(False, True, False, True)

## Creating in other ways

In [21]:
dickens, created = Author.objects.get_or_create(
    name="Charles Dickens",
    dob=1812
)
dickens, created

(<Author #3: Charles Dickens>, True)

In [16]:
wGE,wcreated = dickens.work_set.get_or_create(title="Great Expectations")
wGE,wcreated

(<Work #2: Great Expectations, by <Author #2: Charles Dickens>>, False)

In [18]:
bGE,bcreated = wGE.book_set.get_or_create(title="Great Expectations")
bGE,bcreated

(<Book #3: 'Great Expectations', in <Work #2: Great Expectations, by <Author #2: Charles Dickens>>>,
 False)

In [20]:
tGE,tcreated = bGE.text_set.get_or_create(
    txt = getlink('https://www.gutenberg.org/files/1400/1400-0.txt')
)
tGE,tcreated

(<Text #3: 'ï»¿The Project Gutenberg ...' (1058019 chars), in <Book #3: 'Great Expectations', in <Work #2: Great Expectations, by <Author #2: Charles Dickens>>>>,
 False)